In [ ]:
from dockmate import Protein, Ligand, PocketFinder, VinaDocking
from dockmate.utils import view_molecule, clean_temp_folder, fetch_pdb, fetch_sdf
 
from pathlib import Path
clean_temp_folder()

✅ Temp folder cleaned: C:\Users\MANGAL\Desktop\Docking\dockmate\temp


In [2]:
# Step 1: Prepare the protein
protein = Protein('1HVR.pdb')
protein.prepare()
# protein.save_pdbqt()

# Step 2: Run pocket detection
pf = PocketFinder(protein.pdb_path)
pockets = pf.run()
# pf.save_report()
pockets

[{'rank': 1, 'center': (-9.2421, 16.1006, 27.6026)},
 {'rank': 2, 'center': (-12.5447, 21.7728, 45.5697)}]

In [ ]:
fetch_sdf(cid = "338", save_dir="ligands")
fetch_sdf(cid = "5288826", save_dir="ligands")

# Step 3: Set up ligand folder
ligand_folder = Path("ligands")  # Your folder with SDF files
ligand_files = sorted(ligand_folder.glob("*.sdf"))

### Docks each ligand against a single receptor (at the first pocket).

In [4]:
# Step 4: Loop through each ligand
for ligand_file in ligand_files:
    print(f"\nProcessing: {ligand_file.name}")
    
    # Prepare ligand
    ligand = Ligand(str(ligand_file))
    ligand.prepare(minimize=None)
    # ligand.save_pdbqt()
    
    # Perform docking with first pocket
    print(f"Docking receptor: {protein.pdbqt_path.name}, ligand: {ligand.pdbqt_path.name}")
    
    vina = VinaDocking(
        receptor=protein.pdbqt_path,
        ligand=ligand.pdbqt_path,
        center=pockets[0]['center'],  # Only using first pocket
        size=(40, 40, 40),
        exhaustiveness=8,
        num_modes=9
    )
    vina.run()

    # Save results with unique prefix
    vina.save_results(f"vina_results/{protein.pdbqt_path.stem}_{ligand.pdbqt_path.stem}_pocket_1_docked")



Processing: 338.sdf
Docking receptor: 1HVR.pdbqt, ligand: 338.pdbqt

Processing: 5288826.sdf
Docking receptor: 1HVR.pdbqt, ligand: 5288826.pdbqt


#### Dock each ligand against a single receptor at all the predicted protein pockets


In [5]:
# Step 4: Loop through each ligand
for ligand_file in ligand_files:
    print(f"\nProcessing: {ligand_file.name}")
    
    # Prepare ligand
    ligand = Ligand(str(ligand_file))
    ligand.prepare(minimize=None)
    # ligand.save_pdbqt()
    
    
    for pocket in pockets:
        center = pocket["center"]
        rank = pocket["rank"]

        print(f"Docking receptor: {protein.pdbqt_path.name}, ligand: {ligand.pdbqt_path.name}, pocket: {rank}")

        vina = VinaDocking(
            receptor=protein.pdbqt_path,
            ligand=ligand.pdbqt_path,
            center=center,
            size=(40, 40, 40),
            exhaustiveness=8,
            num_modes=9
        )
        
        vina.run()
        vina.save_results(f"vina_results/{protein.pdbqt_path.stem}_{ligand.pdbqt_path.stem}_pocket_{rank}_docking")



Processing: 338.sdf
Docking receptor: 1HVR.pdbqt, ligand: 338.pdbqt, pocket: 1
Docking receptor: 1HVR.pdbqt, ligand: 338.pdbqt, pocket: 2

Processing: 5288826.sdf
Docking receptor: 1HVR.pdbqt, ligand: 5288826.pdbqt, pocket: 1
Docking receptor: 1HVR.pdbqt, ligand: 5288826.pdbqt, pocket: 2
